#### Turtle Trading

#### Data, Library Load

In [1]:
from tqdm import tqdm 
import pandas as pd
from copy import deepcopy
from dateutil.parser import parse
import datetime as dt
from pykrx import stock
import numpy as np

In [2]:
import sys
sys.path.append('D:/ML/Kis_Api_test/')
from BackTesting_System.backtesting import BackTesting,Utils

In [3]:
path = '../../../mom.csv'
df = pd.read_csv(path,index_col=0)
df = df.drop(df.iloc[:,8:],axis=1)
df.head(1)

,날짜,종가,시가,최고가,최저가,누적거래량,누적거래대금,종목명
0,2022-12-22,10650,9980,11700,9930,28856232,309808795730,바이오노트


In [4]:
df.shape

(2621634, 8)

In [5]:
df = df[df.apply(lambda x: '스팩' not in x['종목명'], axis=1)]
# df = df[df.apply(lambda x: '대성홀딩스' not in x['종목명'], axis=1)]
df.shape

(2605276, 8)

#### function define

In [6]:
# make momentum (10일,20일)
def make_mom(df):
    df = deepcopy(df)
    buf = []
    for _, gdf in tqdm(df.groupby('종목명')):
        gdf['10_mom'] = gdf['종가'] / gdf['종가'].shift(11)*100
        gdf['20_mom'] = gdf['종가'] / gdf['종가'].shift(21)*100
        buf.append(gdf)
    del df
    res = pd.concat(buf)
    return res 

In [7]:
# make tr
def make_tr(df):
    df = deepcopy(df)
    buf = []
    for _, gdf in tqdm(df.groupby('종목명')):
        gdf['con1'] = gdf['최고가'] - gdf['종가'].shift(1)
        gdf['con2'] = gdf['최저가'] - gdf['종가'].shift(1)
        gdf['con1_per'] = (gdf['최고가'] - gdf['종가'].shift(1))/gdf['종가'].shift(1)*100
        gdf['con2_per'] = (gdf['최저가'] - gdf['종가'].shift(1))/gdf['종가'].shift(1)*100
        buf.append(gdf) 
    del df
    res = pd.concat(buf)
    res['con3'] = res['최고가'] - res['최저가']
    res['con3_per'] = ((res['최고가']-res['최저가'])/res['최저가']*100).abs()
    res['con1'] = res['con1'].abs()
    res['con2'] = res['con2'].abs()
    res['con1_per'] = res['con1_per'].abs()
    res['con2_per'] = res['con2_per'].abs()
    res['TR'] = res[['con1','con2','con3']].max(axis=1)
    res['TR_per'] = res[['con1_per','con2_per','con3_per']].max(axis=1)
    res = res.drop(columns=['con1', 'con2', 'con3'], axis=1)
    res = res.drop(columns=['con1_per','con2_per','con3_per'], axis=1)
    return res 

In [8]:
# make atr 21일
def make_atr(df):
    df = deepcopy(df)
    buf = []
    for _, gdf in tqdm(df.groupby('종목명')):
        gdf['ATR'] = gdf['TR'].rolling(21).mean()
        gdf['ATR_per'] = gdf['TR_per'].rolling(21).mean()
        gdf['ATR'].fillna(np.inf, inplace=True)
        gdf['ATR_per'].fillna(np.inf, inplace=True)
        buf.append(gdf)
    del df
    res = pd.concat(buf)
    res = res.drop(columns=['TR'], axis=1)
    return res 

In [9]:
def make_raking(df):
    df = deepcopy(df)
    buf = []
    for _,gdf in tqdm(df.groupby('날짜')):
        gdf['ranking'] = gdf['ATR'].rank(method='min', ascending=True)
        buf.append(gdf)
    del df
    res = pd.concat(buf)
    return res

In [10]:
# make buy
def make_buy(df):
    df = deepcopy(df)
    buf = []
    for _, gdf in tqdm(df.groupby('종목명')):
        gdf['buy1'] = (gdf['20_mom']>100)&(gdf['종가'].rolling(window=10, closed='left').max() < gdf['종가'])  #&(gdf['ranking'] <= 10)     
        #gdf['middle_buy'] = (gdf['종가'].rolling(window=55, closed='left').max() < gdf['종가'])
        #gdf['buy'] = gdf['short_buy'] | gdf['middle_buy']
        buf.append(gdf)
    del df
    res = pd.concat(buf)
    df = deepcopy(res)
    del buf
    buf = []
    for (_, buy), gdf in tqdm(df.groupby(['날짜', 'buy1'])):
        if buy == True:
            gdf['buy2'] = gdf['ATR_per'].rank(method='min', ascending=True) <= 10
        buf.append(gdf)
    res = pd.concat(buf)
    res['buy'] = res['buy1'] & res['buy2']
    res = res.drop(columns=['buy1', 'buy2'], axis=1)
    return res

In [11]:
# make sell
def make_sell(df):
    df = deepcopy(df)
    buf = []
    for _,gdf in tqdm(df.groupby('종목명')):
        con1 = (gdf['10_mom'] > 100)&(gdf['종가'].rolling(window=2,closed='left').min() > gdf['종가'])
        #con2 = (gdf['10_mom'] < 100)&(gdf['종가'].rolling(window=10,closed='left').max() < gdf['종가'])
        gdf['sell'] = con1
        buf.append(gdf)
    del df
    res = pd.concat(buf)

    return res 

#### Do!

In [12]:
df = make_mom(df)

 ... (more hidden) ...


In [13]:
df = make_tr(df)

 ... (more hidden) ...


In [14]:
df = make_atr(df)

 ... (more hidden) ...


In [15]:
# df = make_raking(df)

In [16]:
df = make_buy(df)

 ... (more hidden) ...
 ... (more hidden) ...


In [17]:
df = make_sell(df)

 ... (more hidden) ...


In [18]:
code_kospi = pd.read_csv('../../SRC/current_kospi_codes.csv')
code_kosdaq = pd.read_csv('../../SRC/current_kosdaq_codes.csv')
code = pd.concat([code_kospi,code_kosdaq])
code.rename(columns={'회사명':'종목명'},inplace=True)
code['종목코드'] = code['종목코드'].astype('str')

df_signal = df.merge(code,on='종목명',how='inner')
df_signal.rename(columns={'종목코드':'코드'},inplace=True)
df_signal['구매수량']=1

In [19]:
df_signal['코드'] = df_signal['코드'].apply(lambda x: str(x).zfill(6))

In [20]:
bt = BackTesting()
ut = Utils()

### 일정기간

In [40]:
res = []
for days in [92]:
    first_date = parse(sorted(df_signal['날짜'].unique())[0])
    last_date = parse(sorted(df_signal['날짜'].unique())[-1])
    term = dt.timedelta(days=days)
    term_bias_date = first_date
    while last_date - term_bias_date > term:        
        df_signal_test = df_signal[((df_signal['날짜'] > str(term_bias_date)) & (df_signal['날짜'] < str(term_bias_date + term)))]
        df_stock = stock.get_index_fundamental(str(term_bias_date)[:10], str(term_bias_date + term)[:10], '1001')
        bt.MONEY = 10000000
        first_monry = bt.MONEY
        bt.buy_sell(df=df_signal_test,buy='buy',sell='sell')
        bt.update_information()
        
        a = pd.DataFrame(bt.ASSET).T
        a['코드'] = a.index
        b= df_signal[df_signal['날짜'] == str(term_bias_date + term)][['코드', '종가']]
        df_join = a.merge(b,on='코드', how='inner')
        buf = pd.DataFrame((df_join['종가']* df_join['수량']) - (df_join['평단가']  * df_join['수량'])).sum()[0]

        print('-'*50)
        print('기간:' + str(term_bias_date) + '~' + str(term_bias_date + term))
        print('승률:', round(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0] / bt.ECONO_INFORM.shape[0]*100,2),'%')
        print('승리횟수:' + str(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0]))
        print('전체횟수:' + str(bt.ECONO_INFORM.shape[0]))
        print('수익률', round((bt.ECONO_INFORM['실현손익'].sum() + buf)/first_monry * 100, 2),'%')
        print('시장수익률:', + round(((df_stock.iloc[-1, 0] - df_stock.iloc[0, 0]) / df_stock.iloc[0, 0]) *100, 2), '%')
        print()
        print('-'*50)
        term_bias_date = term_bias_date + term
        res.append(round(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0] / bt.ECONO_INFORM.shape[0]*100,2))
    
    df_signal_test = df_signal[((df_signal['날짜'] > str(term_bias_date)) & (df_signal['날짜'] <= str(last_date)))]
    df_stock = stock.get_index_fundamental(str(term_bias_date)[:10], str(last_date), '1001')
    bt.MONEY = 10000000
    first_monry = bt.MONEY
    bt.buy_sell(df=df_signal_test,buy='buy',sell='sell')
    bt.update_information()
    a = pd.DataFrame(bt.ASSET).T
    a['코드'] = a.index
    b= df_signal[df_signal['날짜'] == str(last_date)][['코드', '종가']]
    df_join = a.merge(b,on='코드', how='inner')
    buf = pd.DataFrame((df_join['종가']* df_join['수량']) - (df_join['평단가']  * df_join['수량'])).sum()[0]
    
    print('-'*50)
    print('기간:' + str(term_bias_date) + '~' + str(last_date))
    print('승률:', round(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0] / bt.ECONO_INFORM.shape[0]*100,2),'%')
    print('승리횟수:' + str(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0]))
    print('전체횟수:' + str(bt.ECONO_INFORM.shape[0]))
    print('수익률:', round((bt.ECONO_INFORM['실현손익'].sum() + buf)/first_monry * 100, 2),'%')
    print('시장수익률:', + round(((df_stock.iloc[-1, 0] - df_stock.iloc[0, 0]) / df_stock.iloc[0, 0]) *100, 2), '%')
    print()
    print('-'*50)
    
    res.append(round(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0] / bt.ECONO_INFORM.shape[0]*100,2))

 ... (more hidden) ...


--------------------------------------------------
기간:2018-01-29 00:00:00~2018-05-01 00:00:00
승률: 4.76 %
승리횟수:2
전체횟수:42
수익률 -1.26 %
시장수익률: -3.19 %

--------------------------------------------------


 ... (more hidden) ...


--------------------------------------------------
기간:2018-05-01 00:00:00~2018-08-01 00:00:00
승률: 1.61 %
승리횟수:1
전체횟수:62
수익률 -4.41 %
시장수익률: -7.92 %

--------------------------------------------------


 ... (more hidden) ...


...

--------------------------------------------------
기간:2022-05-12 00:00:00~2022-08-12 00:00:00
승률: 3.12 %
승리횟수:2
전체횟수:64
수익률 -10.7 %
시장수익률: -0.87 %

--------------------------------------------------


 ... (more hidden) ...


--------------------------------------------------
기간:2022-08-12 00:00:00~2022-11-12 00:00:00
승률: 5.0 %
승리횟수:3
전체횟수:60
수익률 -12.52 %
시장수익률: -1.77 %

--------------------------------------------------


 ... (more hidden) ...


--------------------------------------------------
기간:2022-11-12 00:00:00~2022-12-29 00:00:00
승률: 14.71 %
승리횟수:5
전체횟수:34
수익률: -1.17 %
시장수익률: -9.63 %

--------------------------------------------------


### 5년치 한꺼번에

In [28]:
first_date = parse(sorted(df_signal['날짜'].unique())[0])
last_date = parse(sorted(df_signal['날짜'].unique())[-1])
df_signal_test = df_signal
df_stock = stock.get_index_fundamental(str(first_date)[:10], str(last_date)[:10], '1001')
bt.MONEY = 10000000
first_monry = bt.MONEY
bt.buy_sell(df=df_signal_test,buy='buy',sell='sell')
bt.update_information()


a = pd.DataFrame(bt.ASSET).T
a['코드'] = a.index
b= df_signal[df_signal['날짜'] == str(last_date)][['코드', '종가']]
df_join = a.merge(b,on='코드', how='inner')
res = pd.DataFrame((df_join['종가']* df_join['수량']) - (df_join['평단가']  * df_join['수량'])).sum()[0]

print('-'*50)
print('기간:' + str(first_date) + '~' + str(last_date))
print('승률:', round(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0] / bt.ECONO_INFORM.shape[0]*100,2),'%')
print('승리횟수:' + str(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0]))
print('전체횟수:' + str(bt.ECONO_INFORM.shape[0]))
print('수익률', round((bt.ECONO_INFORM['실현손익'].sum() + res)/first_monry * 100, 2),'%')
print('시장수익률:', + round(((df_stock.iloc[-1, 0] - df_stock.iloc[0, 0]) / df_stock.iloc[0, 0]) *100, 2), '%')
print()
print('-'*50)

 ... (more hidden) ...

2018-01-29
2018-01-30
2018-01-31


 ... (more hidden) ...

2018-02-01
2018-02-02
2018-02-05


 ... (more hidden) ...

...

2022-12-23
2022-12-26
2022-12-26
2022-12-27


 ... (more hidden) ...

2022-12-27
2022-12-28
2022-12-28
2022-12-29


 ... (more hidden) ...


2022-12-29
--------------------------------------------------
기간:2018-01-29 00:00:00~2022-12-29 00:00:00
승률: 37.0 %
승리횟수:441
전체횟수:1192
수익률 11.32 %
시장수익률: -13.92 %

--------------------------------------------------


In [30]:
bt.ECONO_INFORM['실현손익'].sum()

1132259.000000001

In [31]:
bt.ASSET

{'002170': {'평단가': 53140.0, '수량': 10},
 '036190': {'평단가': 29050, '수량': 7},
 '001070': {'평단가': 51800, '수량': 4},
 '016590': {'평단가': 85833.33333333333, '수량': 15},
 '017480': {'평단가': 4995, '수량': 39},
 '043340': {'평단가': 878, '수량': 230},
 '006090': {'평단가': 8160, '수량': 32},
 '000650': {'평단가': 60900, '수량': 4},
 '004370': {'평단가': 357000, '수량': 1},
 '010140': {'평단가': 5220, '수량': 97},
 '007540': {'평단가': 48900, '수량': 7},
 '003100': {'평단가': 133620.0, '수량': 5},
 '072990': {'평단가': 8170, '수량': 43},
 '023460': {'평단가': 2800, '수량': 75},
 '065450': {'평단가': 5680, '수량': 36},
 '032940': {'평단가': 3960, '수량': 51},
 '078070': {'평단가': 16450, '수량': 15},
 '041830': {'평단가': 20200, '수량': 10},
 '044490': {'평단가': 9800, '수량': 20},
 '000540': {'평단가': 3370, '수량': 61}}

### 원하는 날짜 기입

In [21]:
#######여기에 적어주세요##############
first_date = parse('2020-05-01')
last_date = parse('2020-08-01')
#####################################

df_signal_test = df_signal[((df_signal['날짜'] >= str(first_date)) & (df_signal['날짜'] <= str(last_date)))]
df_stock = stock.get_index_fundamental(str(first_date)[:10], str(last_date)[:10], '1001')
bt.MONEY = 10000000
first_monry = bt.MONEY
bt.buy_sell(df=df_signal_test,buy='buy',sell='sell')
bt.update_information()


a = pd.DataFrame(bt.ASSET).T
a['코드'] = a.index
b= df_signal[df_signal['날짜'] == str(last_date)][['코드', '종가']]
df_join = a.merge(b,on='코드', how='inner')
res = pd.DataFrame((df_join['종가']* df_join['수량']) - (df_join['평단가']  * df_join['수량'])).sum()[0]

print('-'*50)
print('기간:' + str(first_date) + '~' + str(last_date))
print('승률:', round(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0] / bt.ECONO_INFORM.shape[0]*100,2),'%')
print('승리횟수:' + str(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0]))
print('전체횟수:' + str(bt.ECONO_INFORM.shape[0]))
print('수익률', round((bt.ECONO_INFORM['실현손익'].sum() + res)/first_monry * 100, 2),'%')
print('시장수익률:', + round(((df_stock.iloc[-1, 0] - df_stock.iloc[0, 0]) / df_stock.iloc[0, 0]) *100, 2), '%')
print()
print('-'*50)

 ... (more hidden) ...

2020-05-04
2020-05-04
2020-05-06


 ... (more hidden) ...

2020-05-06
2020-05-07
2020-05-07
2020-05-08


 ... (more hidden) ...

...

2020-07-27
2020-07-28
2020-07-28
2020-07-29


 ... (more hidden) ...

2020-07-29
2020-07-30
2020-07-30
2020-07-31


 ... (more hidden) ...


2020-07-31
--------------------------------------------------
기간:2020-05-01 00:00:00~2020-08-01 00:00:00
승률: 46.88 %
승리횟수:30
전체횟수:64
수익률 3.98 %
시장수익률: 18.68 %

--------------------------------------------------


In [22]:
bt.JOUNAL[bt.JOUNAL['날짜'] == '2020-05-04']

,날짜,종목코드,체결단가,체결수량,매매비용,매매구분
0,2020-05-04,317240,1333,640,151.0,매수
1,2020-05-04,307750,2191,549,213.0,매수
2,2020-05-04,002840,61900,11,120.0,매수
3,2020-05-04,134380,46400,12,99.0,매수
4,2020-05-04,004080,9580,59,100.0,매수
5,2020-05-04,336060,2372,783,329.0,매수
6,2020-05-04,335870,1999,552,195.0,매수
7,2020-05-04,333430,2366,555,232.0,매수
8,2020-05-04,168490,1970,437,152.0,매수


In [23]:
bt.JOUNAL[bt.JOUNAL['매매구분'] == '매도']

,날짜,종목코드,체결단가,체결수량,매매비용,매매구분
23,2020-05-08,004080,9530,59,1224.0,매도
32,2020-05-11,321260,4020,32,280.0,매도
40,2020-05-12,117580,4810,19,199.0,매도
41,2020-05-12,140910,5379,8,94.0,매도
51,2020-05-13,092130,19300,1,42.0,매도
...,...,...,...,...,...,...
817,2020-07-31,002030,74600,2,325.0,매도
818,2020-07-31,010050,574,130,162.0,매도
819,2020-07-31,323280,2568,33,184.0,매도
820,2020-07-31,272210,8717,21,399.0,매도


In [24]:
bt.ECONO_INFORM

,날짜,매수총합,매도총합,매매비용,실현손익
0,2020-05-04,8993643,0.0,1591.0,0.0
1,2020-05-06,872696,0.0,155.0,0.0
2,2020-05-07,116255,0.0,20.0,0.0
3,2020-05-08,433457,562270,1301.0,-4174.0
4,2020-05-11,239996,128640,321.0,-600.0
...,...,...,...,...,...
59,2020-07-27,1069538,816115,1965.0,-12887.0
60,2020-07-28,212795,173850,415.0,-4558.0
61,2020-07-29,614434,1011903,2311.0,-2203.0
62,2020-07-30,1031580,1090309,2556.0,-5946.0


In [25]:
import plotly.express as px
import numpy as np

In [26]:
test = pd.DataFrame(bt.test)